In [1]:
import sqlite3
import requests
import pandas as pd


In [2]:
server = "https://api3.geo.admin.ch/rest/services/api/MapServer/find?layer=ch.bafu.gewaesserschutz-badewasserqualitaet&sr=4326&geometryFormat=geojson&searchText=CH&searchField=id"

In [3]:
response = requests.get(server)
response.json()

{'results': [{'type': 'Feature',
   'featureId': 'CH10001',
   'bbox': [7.087865, 46.675282, 7.087865, 46.675282],
   'layerBodId': 'ch.bafu.gewaesserschutz-badewasserqualitaet',
   'layerName': 'Badegewässerqualität',
   'id': 'CH10001',
   'geometry': {'type': 'Point', 'coordinates': [7.087865, 46.675282]},
   'properties': {'bwname': 'Plage de Gumefens',
    'groupid': None,
    'nwunitname': 'Lac de la Gruyère',
    'gemeinde': 'Pont-en-Ogoz',
    'canton': 'FR',
    'bwatercat': 'See',
    'url': 'https://www.fr.ch/de/ilfd/news/die-mikrobiologische-wasserqualitaet-der-offiziellen-strandbaeder-ist-gut',
    'baquaimg': None,
    'year_bw': 2022,
    'qualitaet': '1',
    'label': 'Plage de Gumefens'}},
  {'type': 'Feature',
   'featureId': 'CH10004',
   'bbox': [7.110896, 46.927943, 7.110896, 46.927943],
   'layerBodId': 'ch.bafu.gewaesserschutz-badewasserqualitaet',
   'layerName': 'Badegewässerqualität',
   'id': 'CH10004',
   'geometry': {'type': 'Point', 'coordinates': [7.11089

In [4]:
df = pd.json_normalize(response.json()['results'])\
.rename(columns={
    'properties.bwname': 'name',
    'properties.label': 'alternate_name'
})\
.drop(
    columns=[
        'type', 'featureId', 'bbox', 'layerBodId', 'layerName',
        'geometry.type', 'properties.groupid', 'properties.nwunitname', 'properties.gemeinde', 'properties.bwatercat',
        'properties.canton', 'properties.url', 'properties.baquaimg', 'properties.year_bw', 'properties.qualitaet'
    ]
)\
.set_index('id')


In [5]:
# Split coordinates into columns
df[['long', 'lat']] = df['geometry.coordinates'].apply(pd.Series)
locations = df.drop(columns=['geometry.coordinates'])
locations

,name,alternate_name,long,lat
id,,,,
CH10001,Plage de Gumefens,Plage de Gumefens,7.087865,46.675282
CH10004,Gemeinde Strandbad,Gemeinde Strandbad,7.110896,46.927943
CH10007,Plage de Portalban,Plage de Portalban,6.953375,46.922705
CH10009,Nouvelle plage,Nouvelle plage,6.848274,46.857082
CH1013,Strandbad Maur,Strandbad Maur,8.674674,47.345638
...,...,...,...,...
CH7005,Buochs Strandbad,Buochs Strandbad,8.417998,46.981527
CH9001,Strandbad Seeliken,Strandbad Seeliken,8.512601,47.164293
CH9002,Strandbad Brüggli,Strandbad Brüggli,8.495300,47.176508


In [6]:
db = sqlite3.connect("../dataset.sqlite3")
locations.to_sql("locations", db, if_exists="append")

196